In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../..")

from heritageconnector import datastore
from heritageconnector.config import field_mapping
from heritageconnector.best_spacy_pipeline import load_model
from heritageconnector.datastore import es, index
from smg_jobs.smg_loader import preprocess_text_for_ner

import entity_linker

import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2021-03-30 10:00:23,637 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:00:23,637 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


### NERLoader

In [4]:
source_description_field = target_description_field = "data.http://www.w3.org/2001/XMLSchema#description"
target_title_field = "graph.@rdfs:label.@value"
target_alias_field = "graph.@skos:altLabel.@value"
target_type_field = "graph.@skos:hasTopConcept.@value"

record_loader = datastore.RecordLoader("SMG", field_mapping)
ner_loader = datastore.NERLoader(
    record_loader,
    source_es_index = 'heritageconnector_test',
    target_es_index = 'heritageconnector_test',
    source_description_field = source_description_field, 
    target_title_field = target_title_field, 
    target_description_field = target_description_field, 
    target_type_field = target_type_field,
    target_alias_field = target_alias_field,
    entity_types_to_link={"PERSON", "OBJECT", "ORG"},
    text_preprocess_func=preprocess_text_for_ner
)

In [5]:
entity_list = ner_loader.get_list_of_entities_from_es("en_core_web_trf", 100, random_seed=420)
len(entity_list)

2021-03-30 10:00:25,630 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-03-30 10:00:25,630 - heritageconnector.datastore - INFO - Fetching docs and running NER.
spacy tried to use GPU but failed
2021-03-30 10:00:32,187 - hc_nlp.pipeline - INFO - Loading thesaurus from ../../../heritageconnector/../GITIGNORE_DATA/labels_all_unambiguous_types_people_orgs.jsonl
2021-03-30 10:00:33,538 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 1s


0it [00:00, ?it/s]

291

In [6]:
entity_list_with_links = ner_loader.get_link_candidates(candidates_per_entity_mention=10)

2021-03-30 10:00:42,827 - heritageconnector.datastore - INFO - Getting link candidates for each of 291 entities
2021-03-30 10:00:42,827 - heritageconnector.datastore - INFO - Getting link candidates for each of 291 entities


  0%|          | 0/291 [00:00<?, ?it/s]

In [7]:
ent_df = ner_loader.entity_list_as_dataframe
ent_df.columns, ent_df.shape

(Index(['item_uri', 'candidate_rank', 'item_description_with_ent', 'ent_label',
        'ent_text', 'ent_sentence', 'candidate_title', 'candidate_type',
        'candidate_uri', 'link_correct', 'candidate_alias',
        'candidate_description', 'item_description'],
       dtype='object'),
 (1378, 13))

In [8]:
ent_df.head(1)

,item_uri,candidate_rank,item_description_with_ent,ent_label,ent_text,ent_sentence,candidate_title,candidate_type,candidate_uri,link_correct,candidate_alias,candidate_description,item_description
2,https://collection.sciencemuseumgroup.org.uk/objects/co203466,NaN,"Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", [[1923]]",DATE,1923,"Mounted. ""A Sun lit Corner"", 1923",NaN,NaN,NaN,NaN,NaN,NaN,"Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", 1923"


In [9]:
review_df = ner_loader.get_links_data_for_review()
review_df.head()
# review_df.to_csv("./review_data.csv")

,item_uri,candidate_rank,item_description_with_ent,ent_label,ent_text,ent_sentence,candidate_title,candidate_type,candidate_uri,link_correct,candidate_alias,candidate_description,item_description
0,https://collection.sciencemuseumgroup.org.uk/objects/co203466,0.0,"Transfer print by [[Herbert Bairstow]], FRPS. Mounted. ""A Sun lit Corner"", 1923",PERSON,Herbert Bairstow,"Transfer print by Herbert Bairstow, FRPS.",Herbert Bairstow,PERSON,https://collection.sciencemuseumgroup.org.uk/people/cp134294,,,collectionsonline.lacma.org more research needed British pictorialist photographer noted for bromoil transfer prints; awarded FRPS; lived Halifax (1904).,"Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", 1923"
1,https://collection.sciencemuseumgroup.org.uk/objects/co203466,1.0,"Transfer print by [[Herbert Bairstow]], FRPS. Mounted. ""A Sun lit Corner"", 1923",PERSON,Herbert Bairstow,"Transfer print by Herbert Bairstow, FRPS.",S. Herbert,PERSON,https://collection.sciencemuseumgroup.org.uk/people/cp119499,,,Taken from object description,"Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", 1923"
2,https://collection.sciencemuseumgroup.org.uk/objects/co203466,2.0,"Transfer print by [[Herbert Bairstow]], FRPS. Mounted. ""A Sun lit Corner"", 1923",PERSON,Herbert Bairstow,"Transfer print by Herbert Bairstow, FRPS.",Herbert Hoover,PERSON,https://collection.sciencemuseumgroup.org.uk/people/cp48923,,,President of the United States...,"Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", 1923"
3,https://collection.sciencemuseumgroup.org.uk/objects/co203466,3.0,"Transfer print by [[Herbert Bairstow]], FRPS. Mounted. ""A Sun lit Corner"", 1923",PERSON,Herbert Bairstow,"Transfer print by Herbert Bairstow, FRPS.",Herbert Longmuir,PERSON,https://collection.sciencemuseumgroup.org.uk/people/cp119843,,,,"Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", 1923"
4,https://collection.sciencemuseumgroup.org.uk/objects/co203466,4.0,"Transfer print by [[Herbert Bairstow]], FRPS. Mounted. ""A Sun lit Corner"", 1923",PERSON,Herbert Bairstow,"Transfer print by Herbert Bairstow, FRPS.",Herbert Mayo,PERSON,https://collection.sciencemuseumgroup.org.uk/people/cp51684,,,"Paul F. Cranefield, ‘Mayo, Herbert (1796–1852)’, , 2004 [, accessed 12 March 2008] Surgeon and lecturer in anatomy. Became surgeon at the Royal College of Surgeons 1819. Author of 'Anatomical and Physical Commentaries' (1822). Mayo discovered that the seventh cranial nerve is the nerve the activity of which causes the muscles of the face to move—that is, it is the motor nerve of the face. He also discovered that the fifth cranial nerve is the nerve that conveys sensation from the face to the brain—that is, it is the chief sensory nerve of the face.","Transfer print by Herbert Bairstow, FRPS. Mounted. ""A Sun lit Corner"", 1923"


In [8]:
# review_df.to_excel("./review_data_1103.xlsx")

## Train entity linker and push entities to Elasticsearch index

In [12]:
data_path = "../../../GITIGNORE_DATA/NEL/review_data_1103.xlsx"


2021-03-30 10:31:27,227 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:27,227 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:27,227 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:27,227 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


In [13]:
clf = ner_loader.train_entity_linker(data_path)

2021-03-30 10:31:34,161 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:34,161 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:34,161 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:34,161 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:34,161 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-30 10:31:35,050 - heritageconnector.datastore - INFO - Training entity linker...
2021-03-30 10:31:35,050 - heritageconnector.datasto

In [14]:
ner_loader.has_trained_linker

True

In [13]:
ner_loader.load_entities_into_es(linking_confidence_threshold=0.5, force_load_without_linker=True)

2021-03-23 16:23:42,833 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-23 16:23:42,833 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-03-23 16:23:42,954 - heritageconnector.datastore - INFO - Loading 463 entities into heritageconnector_test
2021-03-23 16:23:42,954 - heritageconnector.datastore - INFO - Loading 463 entities into heritageconnector_test
2021-03-23 16:23:43,300 - heritageconnector.datastore - INFO - Loading entity mentions with no link candidates by type...
2021-03-23 16:23:43,300 - heritageconnector.datastore - INFO - Loading entity mentions with no link candidates by type...


  0%|          | 0/8 [00:00<?, ?ent type/s]

2021-03-23 16:23:44,006 - heritageconnector.datastore - INFO - Predicting links for entity mentions with link candidates and loading them in, in batches of 32768...
2021-03-23 16:23:44,006 - heritageconnector.datastore - INFO - Predicting links for entity mentions with link candidates and loading them in, in batches of 32768...


  0%|          | 0/1 [00:00<?, ?batch/s]

2021-03-23 16:23:54,052 - heritageconnector.nlp.nel - DEBUG - Calculating sBERT embeddings... (1/2)
2021-03-23 16:23:56,744 - heritageconnector.nlp.nel - DEBUG - Calculating sBERT embeddings... (2/2)
